# proyecto Web Scraping

PS D:\Estudios_extra\Big_Data_Python\Aplicando_WEB_SCRAPING> Set-ExecutionPolicy -ExecutionPolicy RemoteSigned -Scope Process
PS D:\Estudios_extra\Big_Data_Python\Aplicando_WEB_SCRAPING> venv_scrap\Scripts\activate
(venv_scrap) PS D:\Estudios_extra\Big_Data_Python\Aplicando_WEB_SCRAPING> 
fwd-i-search: _

(venv_scrap) PS D:\Estudios_extra\Big_Data_Python\Aplicando_WEB_SCRAPING> # Crear directorios para la estructura del proyecto
>> mkdir scraper\spiders -Force
>> mkdir data\raw -Force
>> mkdir data\processed -Force
>> mkdir notebooks -Force
>> mkdir tests -Force
>> mkdir logs -Force


(venv_scrap) PS D:\Estudios_extra\Big_Data_Python\Aplicando_WEB_SCRAPING> # Alternativa más sencilla
>> New-Item -Path "scraper\__init__.py" -ItemType "file" -Force
>> New-Item -Path "scraper\spiders\__init__.py" -ItemType "file" -Force

In [1]:
  # pip install requests beautifulsoup4 pandas scrapy selenium lxml

In [2]:
# pip freeze > requirements.txt #Archivo notas de los paquetes instalados

### Paso 1 (define ruta)

La ruta de trabajo

In [33]:
import os #Definir el directorio de trabajo
os.chdir(r'D:\Estudios_extra\Big_Data_Python\Aplicando_WEB_SCRAPING')
os.getcwd() #conocer el directorio de trabajo

'D:\\Estudios_extra\\Big_Data_Python\\Aplicando_WEB_SCRAPING'

In [34]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options  # Importación correcta de Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import time
import random

## Entrar al link

In [35]:
# Mejorar la configuración del navegador
options = Options()
options.add_argument("--start-maximized")  # Maximizar ventana
options.add_argument("--disable-notifications")  # Desactivar notificaciones
options.add_argument("--disable-popup-blocking")  # Desactivar bloqueo de popups
options.add_argument("--disable-infobars")  # Desactivar infobars
# options.add_argument("--headless")  # Modo sin interfaz (descomentar si no necesitas ver el navegador)

service = Service(ChromeDriverManager().install())

# Inicializar el driver
driver = webdriver.Chrome(service=service, options=options)
driver.get("https://siea.midagri.gob.pe/portal/calendario/#")

## Ingresa cultivo y Clase

In [36]:
# Seleccionar Maiz Amarillo Duro
WebDriverWait(driver, 20).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, "span.select2-selection"))
).click()

# Ingresar búsqueda
search_box = WebDriverWait(driver, 10).until(
    EC.visibility_of_element_located((By.CSS_SELECTOR, "input.select2-search__field"))
)
search_box.send_keys("Aceituna", Keys.RETURN)

# Hacer click en el botón "cosecha"
WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.ID, "btnCosecha"))
).click()

time.sleep(2)

## Busca el departamento

In [6]:
try:
    # Configuración inicial
    time.sleep(2)
    action = ActionChains(driver)
    departments = driver.find_elements(By.CSS_SELECTOR, "path.highcharts-point")
    
    # Buscar Lima en los departamentos
    for department in departments:
        try:
            action.move_to_element(department).perform()
            tooltip = driver.find_element(By.CSS_SELECTOR, ".highcharts-tooltip")
            
            if "Moquegua" in tooltip.text.strip():
                # Intentar hacer clic usando diferentes métodos
                try:
                    department.click()
                except:
                    try:
                        action.click(department).perform()
                    except:
                        driver.execute_script("arguments[0].dispatchEvent(new MouseEvent('click', {bubbles: true}));", department)
                break
        except:
            continue
            
except Exception as e:
    print(f"Error: {e}")

### Busca provincia

In [17]:
try:
    # Configuración inicial
    time.sleep(2)
    action = ActionChains(driver)
    departments = driver.find_elements(By.CSS_SELECTOR, "path.highcharts-point")
    
    # Buscar Lima en los departamentos
    for department in departments:
        try:
            action.move_to_element(department).perform()
            tooltip = driver.find_element(By.CSS_SELECTOR, ".highcharts-tooltip")
            
            if "Vegueta" in tooltip.text.strip():
                # Intentar hacer clic usando diferentes métodos
                try:
                    department.click()
                except:
                    try:
                        action.click(department).perform()
                    except:
                        driver.execute_script("arguments[0].dispatchEvent(new MouseEvent('click', {bubbles: true}));", department)
                break
        except:
            continue
            
except Exception as e:
    print(f"Error: {e}")

## Click "Regresar" para volver al inicio

In [9]:
try:    
    # Buscar el botón Regresar usando el selector específico de SVG
    regresar_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, ".highcharts-button-box"))
    )
    
    # Usar ActionChains para mover el mouse y hacer click en Regresar
    action = ActionChains(driver)
    action.move_to_element(regresar_button).click().perform()
    
    # Esperar un momento para que la página se actualice después de hacer clic en Regresar
    time.sleep(2)
    
    # Hacer click en el botón "Cosechas" usando el ID correcto
    try:
        # Primer intento: buscar por ID
        cosecha_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, "btnCosecha"))
        )
        cosecha_button.click()
    except Exception as cosecha_error:
        print(f"Error al buscar por ID: {cosecha_error}")

except Exception as e:
    print(f"Error al hacer click en Regresar: {e}")

In [10]:
# # Lista para almacenar los datos extraídos
# datos_mensuales = []

# # Meses del año
# meses = ['Ene', 'Feb', 'Mar', 'Abr', 'May', 'Jun', 'Jul', 'Ago', 'Set', 'Oct', 'Nov', 'Dic']

# try:
#     # Esperar a que cargue el gráfico
#     WebDriverWait(driver, 10).until(
#         EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".highcharts-column-series .highcharts-point"))
#     )
    
#     # Buscar las barras del gráfico que tienen altura visible
#     barras = driver.find_elements(By.CSS_SELECTOR, ".highcharts-column-series .highcharts-point")
    
#     # Filtrar solo barras visibles (con altura > 0)
#     barras_visibles = []
#     for barra in barras:
#         altura = barra.get_attribute("height")
#         if altura and float(altura) > 0:
#             barras_visibles.append(barra)
    
#     print(f"Se encontraron {len(barras_visibles)} barras visibles de {len(barras)} totales")
    
#     # Obtener el título del gráfico para validar
#     titulo = driver.find_element(By.CSS_SELECTOR, ".highcharts-title").text
#     print(f"Analizando gráfico: {titulo}")
    
#     # Crear diccionario para almacenar datos por mes
#     datos_por_mes = {mes: {"porcentaje": None, "tm": None} for mes in meses}
    
#     # Mover el cursor a cada etiqueta del eje X para determinar qué mes es cada posición
#     etiquetas_x = driver.find_elements(By.CSS_SELECTOR, ".highcharts-xaxis-labels text")
#     print(f"Se encontraron {len(etiquetas_x)} etiquetas en el eje X")
    
#     # Mapear las etiquetas con sus posiciones
#     meses_posiciones = {}
#     for etiqueta in etiquetas_x:
#         texto = etiqueta.text
#         if texto in meses:
#             pos_x = etiqueta.rect['x']
#             meses_posiciones[texto] = pos_x
#             print(f"Etiqueta: {texto}, Posición X: {pos_x}")
    
#     # Analizar cada barra y asociarla con el mes correcto
#     for i, barra in enumerate(barras):
#         try:
#             # Obtener la posición X central de la barra
#             pos_x_barra = barra.rect['x'] + (barra.rect['width'] / 2)
#             altura = float(barra.get_attribute("height") or 0)
            
#             # Encontrar el mes más cercano a esta posición X
#             mes_cercano = None
#             menor_distancia = float('inf')
            
#             for mes, pos_x in meses_posiciones.items():
#                 distancia = abs(pos_x - pos_x_barra)
#                 if distancia < menor_distancia:
#                     menor_distancia = distancia
#                     mes_cercano = mes
            
#             # Solo procesar si la barra tiene altura (es visible)
#             if altura > 0 and mes_cercano:
#                 # Mover el cursor a la barra para mostrar el tooltip
#                 ActionChains(driver).move_to_element(barra).perform()
#                 time.sleep(0.5)  # Esperar a que aparezca el tooltip
                
#                 # Intentar obtener el texto del tooltip
#                 tooltip_elementos = driver.find_elements(By.CSS_SELECTOR, ".highcharts-tooltip text, .highcharts-tooltip-box + text")
                
#                 tooltip_texto = ""
#                 for elem in tooltip_elementos:
#                     texto = elem.text
#                     if texto and ("%" in texto or "tm:" in texto.lower()):
#                         tooltip_texto = texto
#                         break
                
#                 # Si no se encontró con selectores específicos, usar JavaScript
#                 if not tooltip_texto:
#                     tooltip_texto = driver.execute_script("""
#                         const textos = Array.from(document.querySelectorAll('.highcharts-tooltip text tspan'));
#                         return textos.map(t => t.textContent).join('\\n');
#                     """)
                
#                 # Extraer porcentaje y tm del tooltip
#                 import re
#                 porcentaje = None
#                 tm = None
                
#                 # Patrones para extraer porcentaje y tm
#                 porcentaje_match = re.search(r'(\d+\.?\d*)\s*%', tooltip_texto)
#                 tm_match = re.search(r'tm:\s*(\d+\.?\d*)', tooltip_texto)
                
#                 if porcentaje_match:
#                     porcentaje = float(porcentaje_match.group(1))
                
#                 if tm_match:
#                     tm = float(tm_match.group(1))
                
#                 # Guardar datos para este mes
#                 if mes_cercano:
#                     datos_por_mes[mes_cercano] = {
#                         "porcentaje": porcentaje,
#                         "tm": tm,
#                         "altura": altura,
#                         "tooltip": tooltip_texto
#                     }
                    
#                     print(f"Mes {mes_cercano}: Porcentaje={porcentaje}%, TM={tm}, Altura={altura}")
                
#             else:
#                 print(f"Barra {i} sin altura significativa o sin mes asociado")
                
#         except Exception as e:
#             print(f"Error al procesar barra {i}: {e}")
    
#     # Convertir el diccionario a una lista ordenada por los meses
#     datos_mensuales = []
#     for mes in meses:
#         if mes in datos_por_mes:
#             datos = datos_por_mes[mes]
#             datos_mensuales.append({
#                 "mes": mes,
#                 "porcentaje": datos.get("porcentaje"),
#                 "tm": datos.get("tm"),
#                 "altura": datos.get("altura"),
#                 "tooltip": datos.get("tooltip", "")
#             })
    
#     # Mostrar resultados finales
#     print("\nResultados finales ordenados por mes:")
#     print("Mes | Porcentaje | TM | Altura")
#     print("----|------------|----|---------")
#     for dato in datos_mensuales:
#         porcentaje_str = f"{dato['porcentaje']}%" if dato['porcentaje'] is not None else "N/A"
#         tm_str = str(dato['tm']) if dato['tm'] is not None else "N/A"
#         altura_str = str(dato['altura']) if dato['altura'] is not None else "N/A"
#         print(f"{dato['mes']} | {porcentaje_str} | {tm_str} | {altura_str}")

# except Exception as e:
#     print(f"Error general: {e}")

## Extraer los datos

In [11]:
# # Importar la función
# from scraper.utils.extrae_mes import extraer_datos_grafico_calendario
# import pandas as pd

# # Usar la función
# datos = extraer_datos_grafico_calendario(driver)

# # Convertir a DataFrame (matriz)
# if datos and 'datos_mensuales' in datos:
#     # Crear DataFrame a partir de los datos mensuales
#     df = pd.DataFrame(datos['datos_mensuales'])
    
#     # Añadir información del departamento y cultivo
#     df['departamento'] = datos['departamento']
#     df['cultivo'] = datos['cultivo']
    
#     # Reordenar columnas si lo prefieres
#     df = df[['departamento', 'cultivo', 'mes', 'porcentaje', 'tm']]
    
#     # Mostrar como tabla
#     print("\nDatos en formato matriz (DataFrame):")
#     print(df)
    
#     # Opcionalmente guardar en CSV
#     # df.to_csv(f"{datos['departamento']}_{datos['cultivo'].replace(' ', '_')}.csv", index=False)

## Extrae Cuadro distrito

In [47]:
from scraper.utils.extrae_distrito import extraer_datos_distrito_mapa
import pandas as pd
import openpyxl

# Extraer datos del distrito con todos los mensajes visibles
nombre_distrito = "Coronel Gregorio Albarracin Lanchipa"
datos_distrito = extraer_datos_distrito_mapa(driver, nombre_distrito)

# Crear un DataFrame con los datos del distrito
df_distrito = pd.DataFrame({
    'Concepto': ['Superficie (ha)', 'Rendimiento (t/ha)', 'Producción (tm)', 'Participación (%)'],
    'Valor': [
        datos_distrito['superficie_ha'],
        datos_distrito['rendimiento_tha'],
        datos_distrito['produccion_tm'],
        datos_distrito['participacion_porcentaje']
    ],
    'Distrito': [nombre_distrito] * 4  # Agrega el nombre del distrito como una columna
})

# Mostrar el DataFrame
print(f"\nDatos del distrito {datos_distrito['nombre']}:")
print(df_distrito)

# Exportar a Excel
nombre_archivo = f"distrito_{nombre_distrito.replace(' ', '_')}.xlsx"
df_distrito.to_excel(nombre_archivo, index=False)
print(f"\nDatos exportados a: {nombre_archivo}")

Encontrados 176 elementos en el mapa
Tooltip encontrado: Sama
Tooltip encontrado: 
Tooltip encontrado: 
Tooltip encontrado: 
Tooltip encontrado: Inclan
Tooltip encontrado: 
Tooltip encontrado: 
Tooltip encontrado: 
Tooltip encontrado: 
Tooltip encontrado: 
Tooltip encontrado: 
Tooltip encontrado: 
Tooltip encontrado: 
Tooltip encontrado: 
Tooltip encontrado: 
Tooltip encontrado: 
Tooltip encontrado: 
Tooltip encontrado: 
Tooltip encontrado: 
Tooltip encontrado: 
Tooltip encontrado: 
Tooltip encontrado: 
Tooltip encontrado: Sama
Tooltip encontrado: 
Tooltip encontrado: Sama
Tooltip encontrado: 
Tooltip encontrado: Coronel Gregorio Albarracin Lanchipa
¡Distrito encontrado en tooltip!: Coronel Gregorio Albarracin Lanchipa
Error en clic directo: Message: element not interactable
  (Session info: chrome=135.0.7049.96)
Stacktrace:
	GetHandleVerifier [0x003C8073+60707]
	GetHandleVerifier [0x003C80B4+60772]
	(No symbol) [0x001F04FE]
	(No symbol) [0x002397DB]
	(No symbol) [0x0022E215]
	(No symb

## Extrae Meses

In [49]:
# Importar las funciones
from scraper.utils.extrae_mes import extraer_datos_grafico_calendario
from scraper.utils.extrae_cuadro import extraer_datos_resumen_provincia
import pandas as pd
import os
from datetime import datetime

# Extraer datos del gráfico
datos_grafico = extraer_datos_grafico_calendario(driver)

# Extraer datos del resumen
datos_resumen = extraer_datos_resumen_provincia(driver)

# Combinar la información en un solo diccionario
resultados_completos = {
    "departamento": datos_grafico["departamento"],
    "cultivo": datos_grafico["cultivo"],
    "datos_mensuales": datos_grafico["datos_mensuales"],
    "resumen": datos_resumen
}

# Imprimir resultados completos
print("\nResultados completos:")
print(f"Departamento/Provincia: {resultados_completos['departamento']}")
print(f"Cultivo: {resultados_completos['cultivo']}")
print(f"Superficie (ha): {resultados_completos['resumen']['superficie_ha']}")
print(f"Rendimiento (t/ha): {resultados_completos['resumen']['rendimiento_tha']}")
print(f"Producción (tm): {resultados_completos['resumen']['produccion_tm']}")
print(f"Participación (%): {resultados_completos['resumen']['participacion_porcentaje']}")

# Crear directorio de resultados si no existe
directorio_resultados = "resultados_excel"
if not os.path.exists(directorio_resultados):
    os.makedirs(directorio_resultados)

# Generar nombre del archivo basado en departamento, cultivo y fecha
fecha_actual = datetime.now().strftime("%Y%m%d_%H%M%S")
nombre_base = f"{resultados_completos['departamento']}_{resultados_completos['cultivo'].replace(' ', '_')}_{fecha_actual}"

# Crear DataFrame para los datos de resumen
df_resumen = pd.DataFrame({
    'Concepto': ['Departamento/Provincia', 'Cultivo', 'Superficie (ha)', 'Rendimiento (t/ha)', 'Producción (tm)', 'Participación (%)'],
    'Valor': [
        resultados_completos['departamento'],
        resultados_completos['cultivo'],
        resultados_completos['resumen']['superficie_ha'],
        resultados_completos['resumen']['rendimiento_tha'],
        resultados_completos['resumen']['produccion_tm'],
        resultados_completos['resumen']['participacion_porcentaje']
    ]
})

# Convertir a DataFrame para los datos mensuales
if datos_grafico and 'datos_mensuales' in datos_grafico:
    # Crear DataFrame a partir de los datos mensuales
    df_mensual = pd.DataFrame(datos_grafico['datos_mensuales'])
    
    # Añadir información del departamento y cultivo
    df_mensual['departamento'] = datos_grafico['departamento']
    df_mensual['cultivo'] = datos_grafico['cultivo']
    
    # Reordenar columnas
    df_mensual = df_mensual[['departamento', 'cultivo', 'mes', 'porcentaje', 'tm']]
    
    # Mostrar como tabla
    print("\nDatos en formato matriz (DataFrame):")
    print(df_mensual)

    # Exportar a Excel (ambos DataFrames en diferentes hojas)
    nombre_archivo = os.path.join(directorio_resultados, f"{nombre_base}.xlsx")
    
    with pd.ExcelWriter(nombre_archivo) as writer:
        df_resumen.to_excel(writer, sheet_name='Resumen', index=False)
        df_mensual.to_excel(writer, sheet_name='Datos_Mensuales', index=False)
    
    print(f"\nDatos exportados a: {nombre_archivo}")
else:
    # Si no hay datos mensuales, exportar solo el resumen
    nombre_archivo = os.path.join(directorio_resultados, f"{nombre_base}.xlsx")
    df_resumen.to_excel(nombre_archivo, index=False)
    print(f"\nResumen exportado a: {nombre_archivo}")

Mes May: Porcentaje=91.1%, TM=34.0
Mes Jun: Porcentaje=8.9%, TM=None

Datos extraídos para Provincia - Maiz Amarillo Duro:
Mes | Porcentaje | Toneladas Métricas
----|------------|------------------
May | 91.1% | 34.0
Jun | 8.9% | N/A

Datos de resumen para la provincia/departamento: 
Superficie (ha): 15523.0
Rendimiento (t/ha): 9.4
Producción (tm): 145887.0
Participación (%): 99.9

Resultados completos:
Departamento/Provincia: Provincia
Cultivo: Maiz Amarillo Duro
Superficie (ha): 15523.0
Rendimiento (t/ha): 9.4
Producción (tm): 145887.0
Participación (%): 99.9

Datos en formato matriz (DataFrame):
  departamento             cultivo  mes  porcentaje    tm
0    Provincia  Maiz Amarillo Duro  May        91.1  34.0
1    Provincia  Maiz Amarillo Duro  Jun         8.9   NaN

Datos exportados a: resultados_excel\Provincia_Maiz_Amarillo_Duro_20250420_014031.xlsx


### Enlistar mapa

In [8]:
pip install opencv-python pillow matplotlib selenium webdriver-manager pandas

  Using cached opencv_python-4.11.0.86-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
     ---------------------------------------- 0.0/104.6 kB ? eta -:--:--
     ---------------------- ---------------- 61.4/104.6 kB 1.7 MB/s eta 0:00:01
     -------------------------------------- 104.6/104.6 kB 2.0 MB/s eta 0:00:00
  Using cached kiwisolver-1.4.8-cp311-cp311-win_amd64.whl.metadata (6.3 kB)
Using cached opencv_python-4.11.0.86-cp37-abi3-win_amd64.whl (39.5 MB)
   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ---------------------------------------- 2.7/2.7 MB 56.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
   -------------------------- ------------- 5.3/8.1 MB 113.3 MB/s eta 0:00:01
   ---------------------------------------- 8.1/8.1 MB 103.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/222.0 kB ? eta -:--:--
   -------------------------------


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:

def extraer_areas_habilitadas(cultivo="Aceituna"):

    try:
        # Navegar a la página
 
        # Esperar a que el mapa se cargue completamente
        print("Esperando a que el mapa se cargue...")
        time.sleep(5)
        
        # Capturar screenshot para verificación
        driver.save_screenshot(f"{cultivo.lower().replace(' ', '_')}_mapa.png")
        print(f"Captura de pantalla guardada en {cultivo.lower().replace(' ', '_')}_mapa.png")
        
        # Método 1: Extraer información de la sección de distribución departamental
        print("\nBuscando información en la distribución departamental...")
        
        distribucion_info = driver.execute_script("""
            // Buscar todos los elementos con texto
            const elements = document.body.querySelectorAll('*');
            const resultados = [];
            
            // Patrones para buscar textos como "Lima: 1.0%" o "Tacna: 71.1%"
            const pattern = /([A-Za-zÁ-Úá-úÑñ\\s]+):\\s*(\\d+\\.\\d+)%/;
            
            for (const el of elements) {
                if (el.textContent && pattern.test(el.textContent)) {
                    const text = el.textContent.trim();
                    const match = text.match(pattern);
                    
                    if (match) {
                        const [_, departamento, porcentaje] = match;
                        
                        // Verificar que no sea parte del título o encabezado
                        if (!text.includes("Distribución Departamental") && 
                            !text.includes("Calendario de Cosechas") &&
                            departamento.trim() !== "") {
                            
                            resultados.push({
                                departamento: departamento.trim(),
                                porcentaje: parseFloat(porcentaje)
                            });
                        }
                    }
                }
            }
            
            return resultados;
        """)
        
        if distribucion_info and len(distribucion_info) > 0:
            print(f"Se encontraron {len(distribucion_info)} departamentos habilitados en la distribución:")
            
            for info in distribucion_info:
                print(f"- {info['departamento']}: {info['porcentaje']}%")
                
            # Guardar resultados
            df = pd.DataFrame(distribucion_info)
            
            # Ordenar por porcentaje descendente
            df = df.sort_values(by='porcentaje', ascending=False)
            
            # Guardar en CSV
            csv_file = f"{cultivo.lower().replace(' ', '_')}_departamentos.csv"
            df.to_csv(csv_file, index=False, encoding='utf-8')
            print(f"\nResultados guardados en {csv_file}")
            
            return distribucion_info
        else:
            print("No se encontró información en la distribución departamental")
            
            # Método 2: Intentar extraer información directamente del SVG
            print("\nBuscando información directamente en el SVG del mapa...")
            
            svg_info = driver.execute_script("""
                // Buscar elementos SVG coloreados
                const svgPaths = document.querySelectorAll('path[fill]:not([fill="white"]):not([fill="#FFFFFF"]):not([fill="none"])');
                const resultados = [];
                
                for (const path of svgPaths) {
                    const fill = path.getAttribute('fill');
                    
                    // Verificar que no sea blanco o transparente
                    if (fill && fill !== 'white' && fill !== '#FFFFFF' && fill !== 'none' && fill !== 'transparent') {
                        // Para RGB, verificar que no sea muy claro
                        if (fill.startsWith('rgb')) {
                            const rgbMatch = fill.match(/rgb\\((\\d+),\\s*(\\d+),\\s*(\\d+)\\)/);
                            if (rgbMatch) {
                                const [_, r, g, b] = rgbMatch;
                                // Si todos son muy claros, ignorar
                                if (parseInt(r) > 240 && parseInt(g) > 240 && parseInt(b) > 240) {
                                    continue;
                                }
                            }
                        }
                        
                        // Obtener nombre de región si está disponible
                        const dataName = path.getAttribute('data-name') || '';
                        const id = path.getAttribute('id') || '';
                        const title = path.querySelector('title') ? path.querySelector('title').textContent : '';
                        
                        resultados.push({
                            departamento: dataName || title || id || 'Región coloreada',
                            color: fill
                        });
                    }
                }
                
                return resultados;
            """)
            
            if svg_info and len(svg_info) > 0:
                print(f"Se encontraron {len(svg_info)} elementos coloreados en el SVG:")
                
                for info in svg_info:
                    print(f"- {info['departamento']}: {info['color']}")
                
                # Guardar resultados
                df = pd.DataFrame(svg_info)
                
                # Guardar en CSV
                csv_file = f"{cultivo.lower().replace(' ', '_')}_regiones_svg.csv"
                df.to_csv(csv_file, index=False, encoding='utf-8')
                print(f"\nResultados guardados en {csv_file}")
                
                return svg_info
            
            # Método 3: Extraer información de los textos en la página
            print("\nBuscando información en textos relevantes...")
            
            textos_info = driver.execute_script("""
                // Buscar textos relevantes
                const textos = Array.from(document.querySelectorAll('*'))
                    .filter(el => el.textContent && el.textContent.includes('%'))
                    .map(el => el.textContent.trim())
                    .filter(text => /[A-Za-zÁ-Úá-úÑñ\\s]+:[\\s]*\\d+(\\.\\d+)?%/.test(text));
                
                const resultados = [];
                const procesados = new Set();
                
                for (const texto of textos) {
                    const matches = texto.match(/([A-Za-zÁ-Úá-úÑñ\\s]+):[\\s]*(\\d+(\\.\\d+)?)%/g) || [];
                    
                    for (const match of matches) {
                        const partes = match.split(':');
                        if (partes.length === 2) {
                            const departamento = partes[0].trim();
                            const porcentajeStr = partes[1].trim().replace('%', '');
                            
                            // Evitar duplicados
                            if (!procesados.has(departamento) && departamento !== '') {
                                procesados.add(departamento);
                                resultados.push({
                                    departamento: departamento,
                                    porcentaje: parseFloat(porcentajeStr)
                                });
                            }
                        }
                    }
                }
                
                return resultados;
            """)
            
            if textos_info and len(textos_info) > 0:
                print(f"Se encontraron {len(textos_info)} departamentos en textos:")
                
                for info in textos_info:
                    print(f"- {info['departamento']}: {info['porcentaje']}%")
                
                # Guardar resultados
                df = pd.DataFrame(textos_info)
                
                # Ordenar por porcentaje descendente
                df = df.sort_values(by='porcentaje', ascending=False)
                
                # Guardar en CSV
                csv_file = f"{cultivo.lower().replace(' ', '_')}_departamentos_texto.csv"
                df.to_csv(csv_file, index=False, encoding='utf-8')
                print(f"\nResultados guardados en {csv_file}")
                
                return textos_info
            
            print("No se encontró información sobre departamentos habilitados")
            return []
    
    except Exception as e:
        print(f"Error durante la extracción: {str(e)}")
        return []
    
    #finally:
        # Cerrar el navegador
        #driver.quit()

# Función para procesar múltiples cultivos
def procesar_cultivos(cultivos):
    """
    Procesa una lista de cultivos y extrae información para cada uno
    
    Args:
        cultivos: Lista de nombres de cultivos
    """
    resultados = {}
    
    for cultivo in cultivos:
        print(f"\n{'='*50}")
        print(f"PROCESANDO CULTIVO: {cultivo}")
        print(f"{'='*50}")
        
        info = extraer_areas_habilitadas(cultivo)
        resultados[cultivo] = info
    
    # Crear resumen
    print("\n\n=== RESUMEN DE RESULTADOS ===\n")
    
    for cultivo, info in resultados.items():
        num_departamentos = len(info) if info else 0
        print(f"{cultivo}: {num_departamentos} departamentos habilitados")

# Ejecutar para un solo cultivo
if __name__ == "__main__":
    # Para un solo cultivo:
    extraer_areas_habilitadas("Aceituna")  # Cambia al cultivo que necesites
    
    # Para múltiples cultivos, descomenta estas líneas:
    # cultivos = ["Aceituna", "Maiz Amarillo Duro", "Papa", "Arroz"]
    # procesar_cultivos(cultivos)

Esperando a que el mapa se cargue...
Captura de pantalla guardada en aceituna_mapa.png

Buscando información en la distribución departamental...
Se encontraron 24 departamentos habilitados en la distribución:
- Paracas: 60%
- Paracas: 60%
- Paracas: 60%
- Paracas: 60%
- Paracas: 60%
- Paracas: 60%
- Paracas: 60%
- Paracas: 60%
- San Clemente: 35.3%
- San Clemente: 35.3%
- San Clemente: 35.3%
- San Clemente: 35.3%
- San Andres: 1.6%
- San Andres: 1.6%
- San Andres: 1.6%
- San Andres: 1.6%
- Humay: 1.6%
- Humay: 1.6%
- Humay: 1.6%
- Humay: 1.6%
- Otros: 1.4%
- Otros: 1.4%
- Otros: 1.4%
- Otros: 1.4%
Error durante la extracción: name 'pd' is not defined


In [20]:
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
import time
import pandas as pd

def extraer_areas_habilitadas(cultivo="Aceituna"):
    
    try:
        # Esperar a que el mapa se cargue completamente
        print("Esperando a que el mapa se cargue...")
        time.sleep(5)
        
        # Método 1: Simulación de hover sobre el mapa
        print("\nDetectando áreas mediante simulación de hover...")
        
        areas_detectadas = set()
        
        # Encontrar el elemento SVG del mapa
        svg_element = driver.find_element(By.CSS_SELECTOR, "svg")
        
        # Obtener dimensiones del SVG
        svg_size = driver.execute_script("""
            const svg = arguments[0];
            const rect = svg.getBoundingClientRect();
            return {
                width: rect.width,
                height: rect.height,
                x: rect.x,
                y: rect.y
            };
        """, svg_element)
        
        # Crear una cuadrícula para mover el mouse
        grid_size = 20  # Número de puntos a probar en cada dirección
        step_x = svg_size['width'] / grid_size
        step_y = svg_size['height'] / grid_size
        
        actions = ActionChains(driver)
        
        for i in range(grid_size):
            for j in range(grid_size):
                try:
                    # Calcular coordenadas
                    x_offset = int(step_x * i - svg_size['width']/2)
                    y_offset = int(step_y * j - svg_size['height']/2)
                    
                    # Mover el mouse al punto
                    actions.move_to_element_with_offset(svg_element, x_offset, y_offset).perform()
                    
                    # Esperar un momento para que se active el hover
                    time.sleep(0.1)
                    
                    # Detectar tooltips o texto visible
                    texto_hover = driver.execute_script("""
                        // Buscar tooltips visibles
                        const tooltips = document.querySelectorAll('.highcharts-tooltip, .tooltip, [class*="tooltip"]');
                        for (const tooltip of tooltips) {
                            if (tooltip.style.visibility !== 'hidden' && tooltip.style.display !== 'none') {
                                return tooltip.textContent.trim();
                            }
                        }
                        
                        // Buscar elementos con cambios de estilo por hover
                        const elementos_hover = document.querySelectorAll(':hover');
                        for (const elem of elementos_hover) {
                            if (elem.tagName.toLowerCase() === 'path' || elem.tagName.toLowerCase() === 'polygon') {
                                // Buscar texto asociado
                                const title = elem.querySelector('title');
                                if (title) return title.textContent.trim();
                                
                                // Buscar en atributos
                                const dataName = elem.getAttribute('data-name');
                                if (dataName) return dataName;
                                
                                // Buscar texto cercano
                                const bbox = elem.getBBox();
                                const textos = document.querySelectorAll('text');
                                for (const texto of textos) {
                                    const txtBBox = texto.getBBox();
                                    const dist = Math.sqrt(
                                        Math.pow(bbox.x + bbox.width/2 - txtBBox.x - txtBBox.width/2, 2) +
                                        Math.pow(bbox.y + bbox.height/2 - txtBBox.y - txtBBox.height/2, 2)
                                    );
                                    if (dist < 50) return texto.textContent.trim();
                                }
                            }
                        }
                        
                        return null;
                    """)
                    
                    if texto_hover and texto_hover not in areas_detectadas:
                        areas_detectadas.add(texto_hover)
                        print(f"Área detectada: {texto_hover}")
                
                except Exception as e:
                    # Ignorar errores de movimiento
                    pass
        
        # Método 2: Click en diferentes puntos para detectar áreas
        print("\nDetectando áreas mediante clicks simulados...")
        
        for i in range(grid_size):
            for j in range(grid_size):
                try:
                    # Calcular coordenadas
                    x_offset = int(step_x * i - svg_size['width']/2)
                    y_offset = int(step_y * j - svg_size['height']/2)
                    
                    # Hacer click
                    actions.move_to_element_with_offset(svg_element, x_offset, y_offset).click().perform()
                    
                    # Esperar un momento
                    time.sleep(0.1)
                    
                    # Detectar cambios en la página
                    area_info = driver.execute_script("""
                        // Buscar elementos que aparecieron o cambiaron
                        const modals = document.querySelectorAll('.modal, .popup, [class*="dialog"]');
                        for (const modal of modals) {
                            if (modal.style.display !== 'none' && modal.style.visibility !== 'hidden') {
                                return modal.textContent.trim();
                            }
                        }
                        
                        // Buscar cambios en tablas o información
                        const infos = document.querySelectorAll('.info, .details, [class*="selected"]');
                        for (const info of infos) {
                            return info.textContent.trim();
                        }
                        
                        return null;
                    """)
                    
                    if area_info and area_info not in areas_detectadas:
                        areas_detectadas.add(area_info)
                        print(f"Área detectada por click: {area_info}")
                
                except Exception as e:
                    pass
        
        # Método 3: Análisis del DOM completo
        print("\nAnalizando el DOM completo...")
        
        todas_las_areas = driver.execute_script("""
            const resultados = new Set();
            
            // 1. Buscar en todas las tablas
            const tablas = document.querySelectorAll('table');
            for (const tabla of tablas) {
                const filas = tabla.querySelectorAll('tr');
                for (const fila of filas) {
                    const celdas = fila.querySelectorAll('td, th');
                    if (celdas.length > 0) {
                        const texto = celdas[0].textContent.trim();
                        if (texto && texto.length > 1 && !/^\\d+(\\.\\d+)?%?$/.test(texto)) {
                            resultados.add(texto);
                        }
                    }
                }
            }
            
            // 2. Buscar en elementos SVG con eventos
            const svgElements = document.querySelectorAll('path, polygon, rect, circle');
            for (const elem of svgElements) {
                // Guardar el color original
                const originalFill = elem.getAttribute('fill');
                
                // Simular hover cambiando el color
                elem.setAttribute('fill', '#ff0000');
                
                // Restaurar el color original
                setTimeout(() => {
                    elem.setAttribute('fill', originalFill);
                }, 50);
                
                // Buscar cambios en la página
                const cambios = Array.from(document.querySelectorAll('*'))
                    .filter(el => el.textContent !== '')
                    .map(el => el.textContent.trim());
                
                for (const cambio of cambios) {
                    if (cambio && cambio.length > 1 && !/^\\d+(\\.\\d+)?%?$/.test(cambio)) {
                        resultados.add(cambio);
                    }
                }
            }
            
            // 3. Buscar en atributos de datos
            const elementosConDatos = document.querySelectorAll('[data-name], [data-region], [data-area], [data-distrito]');
            for (const elem of elementosConDatos) {
                const atributos = ['data-name', 'data-region', 'data-area', 'data-distrito'];
                for (const attr of atributos) {
                    const valor = elem.getAttribute(attr);
                    if (valor) resultados.add(valor);
                }
            }
            
            // 4. Buscar en elementos ocultos
            const elementosOcultos = document.querySelectorAll('[style*="display: none"], [style*="visibility: hidden"], [hidden]');
            for (const elem of elementosOcultos) {
                const texto = elem.textContent.trim();
                if (texto && texto.length > 1 && !/^\\d+(\\.\\d+)?%?$/.test(texto)) {
                    resultados.add(texto);
                }
            }
            
            return Array.from(resultados);
        """)
        
        for area in todas_las_areas:
            if area not in areas_detectadas:
                areas_detectadas.add(area)
                print(f"Área encontrada en DOM: {area}")
        
        # Método 4: Inspección del código fuente
        print("\nInspeccionando el código fuente...")
        
        source_areas = driver.execute_script("""
            const htmlContent = document.documentElement.innerHTML;
            const resultados = new Set();
            
            // Patrones para encontrar nombres de regiones
            const patrones = [
                /"region":"([^"]+)"/g,
                /"distrito":"([^"]+)"/g,
                /"departamento":"([^"]+)"/g,
                /"provincia":"([^"]+)"/g,
                /"area":"([^"]+)"/g,
                /"name":"([^"]+)"/g,
                /data-name="([^"]+)"/g,
                /data-region="([^"]+)"/g
            ];
            
            for (const patron of patrones) {
                let match;
                while ((match = patron.exec(htmlContent)) !== null) {
                    resultados.add(match[1]);
                }
            }
            
            return Array.from(resultados);
        """)
        
        for area in source_areas:
            if area not in areas_detectadas:
                areas_detectadas.add(area)
                print(f"Área encontrada en código fuente: {area}")
        
        # Filtrar y organizar resultados
        resultados_finales = []
        for area in areas_detectadas:
            # Limpiar y normalizar el nombre
            area_limpia = area.strip()
            if (area_limpia and 
                len(area_limpia) > 1 and 
                not area_limpia.lower() in ['otros', 'otro', 'otras', 'otra'] and
                not area_limpia.isdigit()):
                
                resultados_finales.append({
                    'departamento': area_limpia
                })
        
        # Guardar resultados
        if resultados_finales:
            print(f"\nSe detectaron {len(resultados_finales)} áreas en total:")
            for res in resultados_finales:
                print(f"- {res['departamento']}")
            
            df = pd.DataFrame(resultados_finales)
            csv_file = f"{cultivo.lower().replace(' ', '_')}_areas_completas.csv"
            df.to_csv(csv_file, index=False, encoding='utf-8')
            print(f"\nResultados guardados en {csv_file}")
            
            return resultados_finales
        else:
            print("No se encontraron áreas")
            return []
    
    except Exception as e:
        print(f"Error durante la extracción: {str(e)}")
        import traceback
        traceback.print_exc()
        return []

# Ejecutar
if __name__ == "__main__":
    extraer_areas_habilitadas("Aceituna")

Esperando a que el mapa se cargue...

Detectando áreas mediante simulación de hover...
Área detectada: Humay
Área detectada: Paracas
Área detectada: San Clemente
Área detectada: San Andres
Área detectada: Tupac Amaru Inca
Área detectada: Independencia

Detectando áreas mediante clicks simulados...

Analizando el DOM completo...
Área encontrada en DOM: Perú
            Superficie (ha): 21 581
            Rendimiento (t/ha): 8.9
            Produccion (tm): 191 075
            Ranking: 14° PBI Agricola
Área encontrada en DOM: Dpto.: Tacna
            Superficie (ha): 15 544
            Rendimiento (t/ha): 9.4
            Produccion (tm): 145 971
            Participación (%): 71.1
Área encontrada en DOM: Prov.: Tacna
          Superficie (ha): 15 544
          Rendimiento (t/ha): 9.4
          Produccion (tm): 145 971
          Participación (%): 71.1
Área encontrada en DOM: Calendario
    
    
    

    

    
    
    

    
    
    

    
    
    
    

    
    
    
    

    
  

In [27]:
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
import time
import pandas as pd
import numpy as np

def extraer_areas_habilitadas(grid_size=40, wait_time=0.1, segunda_pasada=True):
    """
    Extrae áreas habilitadas mediante simulación de hover
    
    Args:
        grid_size: Resolución de la cuadrícula (mayor número = más puntos de prueba)
        wait_time: Tiempo de espera entre movimientos (segundos)
        segunda_pasada: Si True, realiza una segunda pasada adaptativa en zonas sin detección
    """
    
    try:
        # Esperar a que el mapa se cargue completamente
        print("Esperando a que el mapa se cargue...")
        time.sleep(5)
        
        # Encontrar el elemento SVG del mapa
        svg_element = driver.find_element(By.CSS_SELECTOR, "svg")
        
        # Obtener dimensiones del SVG
        svg_size = driver.execute_script("""
            const svg = arguments[0];
            const rect = svg.getBoundingClientRect();
            return {
                width: rect.width,
                height: rect.height,
                x: rect.x,
                y: rect.y
            };
        """, svg_element)
        
        print(f"Dimensiones del SVG: {svg_size['width']}x{svg_size['height']}")
        print(f"Usando cuadrícula de {grid_size}x{grid_size} puntos")
        
        # Primera pasada
        areas_detectadas = set()
        puntos_probados = []
        
        # Crear una cuadrícula para mover el mouse
        step_x = svg_size['width'] / grid_size
        step_y = svg_size['height'] / grid_size
        
        actions = ActionChains(driver)
        
        # Primera pasada: Exploración sistemática
        print("\nRealizando primera pasada de detección...")
        for i in range(grid_size):
            for j in range(grid_size):
                try:
                    # Calcular coordenadas
                    x_offset = int(step_x * i - svg_size['width']/2)
                    y_offset = int(step_y * j - svg_size['height']/2)
                    
                    # Guardar punto probado
                    puntos_probados.append((x_offset, y_offset))
                    
                    # Mover el mouse al punto
                    actions.move_to_element_with_offset(svg_element, x_offset, y_offset).perform()
                    
                    # Esperar para que se active el hover
                    time.sleep(wait_time)
                    
                    # Detectar área activa
                    area_info = driver.execute_script("""
                        // Buscar tooltips visibles
                        const tooltips = document.querySelectorAll('.highcharts-tooltip, .tooltip, [class*="tooltip"]');
                        for (const tooltip of tooltips) {
                            if (tooltip.style.visibility !== 'hidden' && tooltip.style.display !== 'none') {
                                return {
                                    texto: tooltip.textContent.trim(),
                                    tipo: 'tooltip'
                                };
                            }
                        }
                        
                        // Buscar elementos con hover activo
                        const elementos_hover = document.querySelectorAll(':hover');
                        for (const elem of elementos_hover) {
                            if (elem.tagName.toLowerCase() === 'path' || elem.tagName.toLowerCase() === 'polygon') {
                                // Buscar texto asociado
                                const title = elem.querySelector('title');
                                if (title) return {
                                    texto: title.textContent.trim(),
                                    tipo: 'title'
                                };
                                
                                // Buscar en atributos
                                const dataName = elem.getAttribute('data-name');
                                if (dataName) return {
                                    texto: dataName,
                                    tipo: 'data-name'
                                };
                                
                                // Buscar texto cercano
                                const bbox = elem.getBBox();
                                const textos = document.querySelectorAll('text');
                                for (const texto of textos) {
                                    const txtBBox = texto.getBBox();
                                    const dist = Math.sqrt(
                                        Math.pow(bbox.x + bbox.width/2 - txtBBox.x - txtBBox.width/2, 2) +
                                        Math.pow(bbox.y + bbox.height/2 - txtBBox.y - txtBBox.height/2, 2)
                                    );
                                    if (dist < 50) return {
                                        texto: texto.textContent.trim(),
                                        tipo: 'texto-cercano'
                                    };
                                }
                            }
                        }
                        
                        return null;
                    """)
                    
                    if area_info and area_info['texto'] and area_info['texto'] not in areas_detectadas:
                        areas_detectadas.add(area_info['texto'])
                        print(f"Área detectada: {area_info['texto']} (tipo: {area_info['tipo']})")
                
                except Exception as e:
                    # Ignorar errores de movimiento
                    pass
        
        # Segunda pasada adaptativa (opcional)
        if segunda_pasada:
            print("\nRealizando segunda pasada adaptativa en zonas sin detección...")
            
            # Identificar zonas sin detección
            matriz_deteccion = np.zeros((grid_size, grid_size))
            
            # Marcar zonas con detección
            for i in range(grid_size):
                for j in range(grid_size):
                    x_offset = int(step_x * i - svg_size['width']/2)
                    y_offset = int(step_y * j - svg_size['height']/2)
                    
                    # Verificar si algún área fue detectada cerca de este punto
                    punto_con_deteccion = False
                    for area in areas_detectadas:
                        # Aquí podrías implementar una lógica más sofisticada
                        # para determinar si un punto tiene detección cercana
                        # Por ahora, simplemente marcamos puntos con detección
                        if len(areas_detectadas) > 0:
                            punto_con_deteccion = True
                            break
                    
                    if punto_con_deteccion:
                        matriz_deteccion[i, j] = 1
            
            # Realizar búsqueda detallada en zonas sin detección
            grid_size_detallado = 5  # Resolución más alta para zonas específicas
            
            for i in range(grid_size):
                for j in range(grid_size):
                    if matriz_deteccion[i, j] == 0:  # Zona sin detección
                        # Calcular área de búsqueda
                        x_start = int(step_x * i - svg_size['width']/2)
                        y_start = int(step_y * j - svg_size['height']/2)
                        x_end = int(step_x * (i + 1) - svg_size['width']/2)
                        y_end = int(step_y * (j + 1) - svg_size['height']/2)
                        
                        # Búsqueda detallada en esta zona
                        step_x_detallado = (x_end - x_start) / grid_size_detallado
                        step_y_detallado = (y_end - y_start) / grid_size_detallado
                        
                        for di in range(grid_size_detallado):
                            for dj in range(grid_size_detallado):
                                x_offset = int(x_start + step_x_detallado * di)
                                y_offset = int(y_start + step_y_detallado * dj)
                                
                                try:
                                    actions.move_to_element_with_offset(svg_element, x_offset, y_offset).perform()
                                    time.sleep(wait_time * 1.5)  # Más tiempo para áreas pequeñas
                                    
                                    # Detectar área activa (mismo código que antes)
                                    area_info = driver.execute_script("""
                                        // [Mismo código JavaScript que en la primera pasada]
                                        return null;
                                    """)
                                    
                                    if area_info and area_info['texto'] and area_info['texto'] not in areas_detectadas:
                                        areas_detectadas.add(area_info['texto'])
                                        print(f"Área detectada (2da pasada): {area_info['texto']}")
                                
                                except Exception as e:
                                    pass
        
        # Filtrar y organizar resultados
        resultados_finales = []
        for area in areas_detectadas:
            # Limpiar y normalizar el nombre
            area_limpia = area.strip()
            if (area_limpia and 
                len(area_limpia) > 1 and 
                not area_limpia.lower() in ['otros', 'otro', 'otras', 'otra'] and
                not area_limpia.isdigit()):
                
                resultados_finales.append({
                    'departamento': area_limpia
                })
        
        # Ordenar resultados
        resultados_finales = sorted(resultados_finales, key=lambda x: x['departamento'])
        
        # Guardar resultados
        if resultados_finales:
            print(f"\nSe detectaron {len(resultados_finales)} áreas en total:")
            for res in resultados_finales:
                print(f"- {res['departamento']}")
            
            df = pd.DataFrame(resultados_finales)
            csv_file = "areas_detectadas.csv"
            df.to_csv(csv_file, index=False, encoding='utf-8')
            print(f"\nResultados guardados en {csv_file}")
            
            return resultados_finales
        else:
            print("No se encontraron áreas")
            return []
    
    except Exception as e:
        print(f"Error durante la extracción: {str(e)}")
        import traceback
        traceback.print_exc()
        return []

# Ejemplo de uso con diferentes configuraciones
if __name__ == "__main__":
    # Configuración estándar
    extraer_areas_habilitadas()
    
    # Configuración para áreas pequeñas
    # extraer_areas_habilitadas(grid_size=40, wait_time=0.2, segunda_pasada=True)
    
    # Configuración rápida sin segunda pasada
    # extraer_areas_habilitadas(grid_size=15, wait_time=0.05, segunda_pasada=False)

Esperando a que el mapa se cargue...
Dimensiones del SVG: 569x718
Usando cuadrícula de 40x40 puntos

Realizando primera pasada de detección...
Área detectada: Jayanca (tipo: tooltip)
Área detectada: Morrope (tipo: tooltip)
Área detectada: Olmos (tipo: tooltip)
Área detectada: Salas (tipo: tooltip)
Área detectada: Lambayeque (tipo: tooltip)
Área detectada: San Jose (tipo: tooltip)
Área detectada: Pacora (tipo: tooltip)
Área detectada: Mochumi (tipo: tooltip)
Área detectada: Tucume (tipo: tooltip)
Área detectada: Motupe (tipo: tooltip)
Área detectada: Chochope (tipo: tooltip)

Realizando segunda pasada adaptativa en zonas sin detección...

Se detectaron 11 áreas en total:
- Chochope
- Jayanca
- Lambayeque
- Mochumi
- Morrope
- Motupe
- Olmos
- Pacora
- Salas
- San Jose
- Tucume

Resultados guardados en areas_detectadas.csv


In [37]:
# Importar la función
from scraper.utils.extraer_mapa import extraer_areas_habilitadas

# O usar con configuración personalizada
areas = extraer_areas_habilitadas(driver, grid_size=30, wait_time=0.15, segunda_pasada=True)

Esperando a que el mapa se cargue...
Dimensiones del SVG: 569x718
Usando cuadrícula de 30x30 puntos

Realizando primera pasada de detección...
Área detectada: San Andres (tipo: tooltip)
Área detectada: Paracas (tipo: tooltip)
Área detectada: San Clemente (tipo: tooltip)
Área detectada: Tupac Amaru Inca (tipo: tooltip)
Área detectada: Independencia (tipo: tooltip)
Área detectada: Humay (tipo: tooltip)

Realizando segunda pasada adaptativa en zonas sin detección...

Se detectaron 6 áreas en total:
- Humay
- Independencia
- Paracas
- San Andres
- San Clemente
- Tupac Amaru Inca

Resultados guardados en areas_detectadas.csv
